In [ ]:
# ==============================
# ПУНКТ 1: Загрузка и дескриптивный анализ
# ==============================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Отключаем предупреждения для красоты
import warnings
warnings.filterwarnings('ignore')

# 1. Загрузка датасета Wine (встроен в sklearn, но мы загрузим как DataFrame)
from sklearn.datasets import load_wine
data = load_wine()

df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target  # это настоящие классы вин (3 сорта) — потом уберём для кластеризации

print("Первые 5 строк датасета:")
display(df.head())

# Размерность
print(f"\nРазмерность датасета: {df.shape[0]} объектов, {df.shape[1]} признаков (включая target)")

# Типы признаков и наличие пропусков
print("\nИнформация о признаках:")
display(df.info())

print("\nПропущенные значения:")
display(df.isnull().sum())

# Удаляем колонку с истинными метками для «честной» кластеризации
df_clust = df.drop('target', axis=1).copy()

print(f"\nПосле удаления столбца target остаётся {df_clust.shape[1]} признаков для кластеризации")

# Описательная статистика
print("\nОписательная статистика числовых признаков:")
display(df_clust.describe().round(2))

# ==============================
# Оценка распределения переменных (гистограммы + тест на нормальность)
# ==============================
plt.figure(figsize=(16, 12))
for i, col in enumerate(df_clust.columns, 1):
    plt.subplot(4, 4, i)
    sns.histplot(df_clust[col], kde=True, stat="density", linewidth=1.2)
    plt.title(col, fontsize=12)
    plt.xlabel('')
    plt.ylabel('')
plt.tight_layout()
plt.suptitle("Гистограммы признаков с ядерной оценкой плотности", fontsize=16, y=1.02)
plt.show()

# Тест Шапиро-Уилка на нормальность (p-value < 0.05 → отвергаем нормальность)
print("\nТест Шапиро-Уилка на нормальность (p-value < 0.05 → не нормальное):")
for col in df_clust.columns:
    stat, p = stats.shapiro(df_clust[col])
    print(f"{col:25} | p-value = {p:.2e} → {'НЕ нормальное' if p < 0.05 else 'нормальное'}")

# ==============================
# Оценка информационной значимости признаков
# ==============================
# Корреляционная матрица (для выявления мультиколлинеарности и самых информативных признаков)
plt.figure(figsize=(12, 10))
corr = df_clust.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=.5, cbar_kws={"shrink": .8})
plt.title("Корреляционная матрица признаков")
plt.show()

# Признаки с высокой корреляцией (>0.7 по модулю)
high_corr = np.where(np.abs(corr) > 0.7)
high_corr_pairs = [(corr.index[x], corr.columns[y], corr.iloc[x,y]) 
                   for x,y in zip(*high_corr) if x != y and x < y]
print("\nПары признаков с |корреляция| > 0.7:")
for pair in high_corr_pairs:
    print(f"  {pair[0]:25} — {pair[1]:25} : {pair[2]:.3f}")

# ==============================
# Проверка условий применения кластеризации
# ==============================
print("\n" + "="*60)
print("ПРОВЕРКА УСЛОВИЙ ПРИМЕНЕНИЯ КЛАСТЕРИЗАЦИИ")
print("="*60)

print("1. Отсутствие классов в данных для кластеризации:")
print("   → Да, мы удалили столбец 'target'. Алгоритм будет работать без подсказок.")

print("\n2. Осмысленность кластеризации:")
print("   → Датасет содержит химические характеристики вин трёх сортов.")
print("     Ожидается, что вина одного сорта будут иметь похожий химический состав,")
print("     поэтому кластеризация по составу имеет практический смысл.")

print("\n3. Выбросы (визуально + IQR-метод):")
outliers_info = []
plt.figure(figsize=(16, 10))
for i, col in enumerate(df_clust.columns, 1):
    plt.subplot(4, 4, i)
    sns.boxplot(x=df_clust[col])
    plt.title(col)
    
    Q1 = df_clust[col].quantile(0.25)
    Q3 = df_clust[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5*IQR
    upper = Q3 + 1.5*IQR
    outliers_count = ((df_clust[col] < lower) | (df_clust[col] > upper)).sum()
    outliers_info.append((col, outliers_count))
plt.tight_layout()
plt.suptitle("Box-plots для выявления выбросов", fontsize=16, y=1.02)
plt.show()

print("Количество выбросов по правилу 1.5×IQR:")
for col, cnt in outliers_info:
    print(f"  {col:25}: {cnt} выбросов")

print("\nВывод по выбросам:")
print("   → Есть небольшое количество выбросов (в основном в total_phenols, flavanoids,")
print("     nonflavanoid_phenols, proanthocyanins, color_intensity, hue, proline).")
print("     Их немного (<5% от данных), поэтому на данном этапе оставляем как есть.")
print("     При необходимости можно будет обработать позже (лог-преобразование или удаление).")

print("\nИТОГО: Все основные условия для применения кластеризации выполнены.")